# Lucid Sonic Dreams - Google Colab

Generate stunning AI music visualizations using StyleGAN3.

**Performance Features:**
- StyleGAN3 (preferred, better maintained)
- FP16 inference (2x faster on GPU)
- torch.compile() support (PyTorch 2.0+)
- Optimized vector math with NumPy

**Requirements:**
- GPU runtime (Runtime > Change runtime type > T4 GPU)
- Audio file (.mp3, .wav)

## 1. Setup & Installation

Run this cell to install all dependencies. Takes about 2-3 minutes.

In [ ]:
# Check GPU availability
!nvidia-smi

# Clone and install lucid-sonic-dreams
!git clone https://github.com/RayceRossum/lucid-sonic-dreams.git /content/lucid-sonic-dreams
%cd /content/lucid-sonic-dreams
!pip install -e . -q

# Install PyTorch with CUDA (if not already installed)
!pip install torch torchvision torchaudio -q
!pip install ninja -q  # For faster StyleGAN compilation

print("\n" + "="*50)
print("Installation complete!")
print("="*50)

## 2. Setup StyleGAN3

The library automatically clones StyleGAN3 when needed, but you can pre-setup it here.

In [ ]:
from lucidsonicdreams import setup_stylegan, get_stylegan_base_dir

# Pre-setup StyleGAN3 (this is done automatically, but you can do it manually)
base_dir = get_stylegan_base_dir()
stylegan_path = setup_stylegan(base_dir, use_stylegan3=True)
print(f"StyleGAN3 ready at: {stylegan_path}")

## 3. Mount Google Drive (Optional)

Mount to access your audio files and save outputs.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
OUTPUT_DIR = '/content/drive/MyDrive/LucidSonicDreams/outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

## 4. Upload Audio File

In [ ]:
from google.colab import files

print("Upload your audio file (.mp3 or .wav):")
uploaded = files.upload()
AUDIO_FILE = list(uploaded.keys())[0]
print(f"\nUploaded: {AUDIO_FILE}")

## 5. Check System & Available Styles

In [ ]:
import torch
from lucidsonicdreams import (
    show_styles, is_colab, suggest_batch_size,
    get_gpu_memory_mb, get_free_gpu_memory_mb
)

print("=" * 50)
print("SYSTEM INFO")
print("=" * 50)
print(f"Running on Colab: {is_colab()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {get_gpu_memory_mb():.0f} MB total")
    print(f"GPU Memory Free: {get_free_gpu_memory_mb():.0f} MB")
print(f"torch.compile available: {hasattr(torch, 'compile')}")

print("\n" + "=" * 50)
print("RECOMMENDED SETTINGS")
print("=" * 50)
print(f"Batch size for 512px: {suggest_batch_size(512)}")
print(f"Batch size for 1024px: {suggest_batch_size(1024)}")

print("\n" + "=" * 50)
print("AVAILABLE STYLES")
print("=" * 50)
show_styles()

## 6. Generate Video (Quick Test)

Generate a short test video with optimized settings.

In [ ]:
from lucidsonicdreams import LucidSonicDream

# Configuration
STYLE = 'abstract art'  # Try: 'modern art', 'textures', etc.
DURATION = 30           # Seconds (start short for testing)
RESOLUTION = 512        # 512 is faster, 1024 for higher quality
FPS = 30                # Frames per second

# Create visualizer (will auto-setup StyleGAN3 and enable optimizations)
L = LucidSonicDream(
    song=AUDIO_FILE,
    style=STYLE
)

# Generate video
OUTPUT_FILE = f"{os.path.splitext(AUDIO_FILE)[0]}_visual.mp4"

L.hallucinate(
    file_name=OUTPUT_FILE,
    resolution=RESOLUTION,
    fps=FPS,
    duration=DURATION,
    batch_size=1,
    speed_fpm=12,
    motion_react=0.5,
    pulse_react=0.5,
)

print(f"\nVideo saved: {OUTPUT_FILE}")

## 7. Preview & Download

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Display video
mp4 = open(OUTPUT_FILE, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=512 controls><source src="{data_url}" type="video/mp4"></video>')

In [ ]:
# Download
files.download(OUTPUT_FILE)

## 8. Advanced: Full Quality Render

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(
    song=AUDIO_FILE,
    style='abstract art',
)

L.hallucinate(
    file_name='high_quality.mp4',
    
    # Video settings
    resolution=1024,      # High resolution
    fps=60,               # Smooth 60fps
    duration=None,        # Full song
    batch_size=1,
    
    # Motion settings
    speed_fpm=12,
    motion_react=0.5,
    motion_randomness=0.5,
    motion_percussive=False,
    motion_harmonic=True,
    
    # Pulse settings
    pulse_react=0.5,
    pulse_percussive=True,
    pulse_harmonic=False,
    
    # Visual variety
    truncation=1.0,
    
    # Effects
    contrast_strength=0.5,
    flash_strength=0.3,
)

## 9. Custom StyleGAN Model

In [ ]:
# Use your own .pkl model
CUSTOM_MODEL = '/content/drive/MyDrive/models/your_model.pkl'

L = LucidSonicDream(
    song=AUDIO_FILE,
    style=CUSTOM_MODEL
)

L.hallucinate(
    file_name='custom_output.mp4',
    resolution=512,
    fps=30,
    duration=30,
)

## 10. Cleanup

In [ ]:
import torch
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Clean temp files
!rm -rf /tmp/lsd_*.mp4 /tmp/lsd_*.wav

print("Cleanup complete!")
if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info()
    print(f"Free GPU memory: {free / 1024**2:.0f} MB")

## Performance Tips

**Speed Optimizations (automatic):**
- FP16 inference (2x faster)
- torch.compile() on PyTorch 2.0+
- torch.inference_mode()
- Pre-allocated arrays
- Vectorized NumPy operations

**Manual Tuning:**
- Lower resolution (512 vs 1024) for 4x faster generation
- Lower FPS (24 vs 60) for 2.5x faster
- Use `batch_size=2` if you have >12GB VRAM

**Troubleshooting:**
- OOM: Reduce resolution or batch_size
- Slow: Ensure GPU runtime is enabled
- Codec errors: Library auto-tries aac, mp3 codecs